In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.stats import pearsonr
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
from pyro.infer import Predictive
from pyro.infer import MCMC, NUTS

In [2]:
#import creep data
creep_df = pd.read_csv('Ti_alloys_dataset.csv')
creep_df

,Ti,Al,V,Fe,C,N,H,O,Sn,Nb,...,Cr,Solution treated temp(cel),ST time (h),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc),creep_rupture_life
0,87.8750,6.75,4.50,0.40,0.100,0.050,0.1250,0.20,0.00,0.0,...,0.0,690,4.0,1050,0.5,700,319.0,2.090000e-03,15.80,0.01000
1,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,319.0,3.240000e-06,0.00,0.01167
2,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,250.0,4.400000e-05,0.00,0.03050
3,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,500,520.0,4.430000e-05,0.00,0.04620
4,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,700,319.0,2.130000e-03,0.00,0.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,85.9870,6.20,0.00,0.00,0.021,0.004,0.0080,0.03,1.95,0.0,...,0.0,900,1.0,580,8.0,520,310.0,1.200000e-04,0.00,1160.00000
173,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,940,4.0,455,379.0,9.160000e-07,0.00,1619.00000
174,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,1030,0.5,455,379.0,1.270000e-06,0.00,1744.00000
175,89.4100,6.00,4.00,0.25,0.080,0.050,0.0100,0.20,0.00,0.0,...,0.0,0,0.0,0,0.0,538,103.0,9.000000e-07,22.30,4681.00000


In [3]:
creep_df.head()

,Ti,Al,V,Fe,C,N,H,O,Sn,Nb,...,Cr,Solution treated temp(cel),ST time (h),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc),creep_rupture_life
0,87.875,6.75,4.50,0.40,0.100,0.050,0.125,0.20,0.0,0.0,...,0.0,690,4.0,1050,0.5,700,319.0,0.002090,15.8,0.01000
1,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,600,319.0,0.000003,0.0,0.01167
2,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,600,250.0,0.000044,0.0,0.03050
3,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,500,520.0,0.000044,0.0,0.04620
4,87.810,6.61,4.23,1.18,0.026,0.011,0.003,0.13,0.0,0.0,...,0.0,0,0.0,0,0.0,700,319.0,0.002130,0.0,0.05000


In [4]:
class BNN(PyroModule):
    def __init__(self, in_dim=12, out_dim=1, hid_dim=10, n_hid_layers=5, prior_scale=5.):
        super().__init__()

        self.activation = nn.Tanh()  # could also be ReLU or LeakyReLU
        assert in_dim > 0 and out_dim > 0 and hid_dim > 0 and n_hid_layers > 0  # make sure the dimensions are valid

        # Define the layer sizes and the PyroModule layer list
        self.layer_sizes = [in_dim] + n_hid_layers * [hid_dim] + [out_dim]
        layer_list = [PyroModule[nn.Linear](self.layer_sizes[idx - 1], self.layer_sizes[idx]) for idx in
                      range(1, len(self.layer_sizes))]
        self.layers = PyroModule[torch.nn.ModuleList](layer_list)

        for layer_idx, layer in enumerate(self.layers):
            layer.weight = PyroSample(dist.Normal(0., prior_scale * np.sqrt(2 / self.layer_sizes[layer_idx])).expand(
                [self.layer_sizes[layer_idx + 1], self.layer_sizes[layer_idx]]).to_event(2))
            layer.bias = PyroSample(dist.Normal(0., prior_scale).expand([self.layer_sizes[layer_idx + 1]]).to_event(1))

    def forward(self, x, y=None):
        # x = x.reshape(-1, 1)
        x = self.activation(self.layers[0](x))  # input --> hidden
        for layer in self.layers[1:-1]:
            x = self.activation(layer(x))  # hidden --> hidden
        mu = self.layers[-1](x).squeeze()  # hidden --> output
        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))  # infer the response noise

        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mu, sigma * sigma), obs=y)
        return mu



In [5]:
rm_state = 123
test_size = 0.6

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:12]), np.array(creep_df.iloc[:,12]), shuffle=True, test_size=test_size, random_state=rm_state)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_test = scaler.transform(X_test)

In [6]:
idx = np.arange(len(y))

train_ratio = 0.1
X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [7]:
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []
num_training_data=[]

pcc_random = []
r2_random = []
num_iteration = []

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

In [8]:
n_iter = 6
test_list = []

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        
        q_points_var = np.argpartition(y_pred_unc_pool_var, -10)[-10:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]
        

    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")
    
    num_training_data.append(len(idx_train_var))
    
    model = BNN(hid_dim=10, n_hid_layers=3, prior_scale=1)
    nuts_kernel = NUTS(model, jit_compile=False)
    mcmc = MCMC(nuts_kernel, num_samples=100)
    mcmc.run(torch.Tensor(X_train_var), torch.Tensor(y_train_var))
    predictive_var = Predictive(model=model, posterior_samples=mcmc.get_samples())
    preds_var = predictive_var(torch.Tensor(X_test))

    # mean and standard deviation of the test dataset
    y_pred_test_var = preds_var['obs'].T.detach().numpy().mean(axis=1)
    y_std_test_var = preds_var['obs'].T.detach().numpy().std(axis=1)

    preds_pool_var = predictive_var(torch.Tensor(X[idx_pool_var]))
    y_pred_unc_pool_var = preds_pool_var['obs'].T.detach().numpy().std(axis=1)

    print('PCC_test variance', pearsonr(np.squeeze(y_test), np.squeeze(y_pred_test_var))[0])
    print('R2_test variance', r2_score(np.squeeze(y_test), np.squeeze(y_pred_test_var)))
    print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    print('MAE', np.mean(abs(y_test - y_pred_test_var)))

    pcc_variance.append(pearsonr(np.squeeze(y_test), np.squeeze(y_pred_test_var))[0])
    r2_variance.append(r2_score(np.squeeze(y_test), np.squeeze(y_pred_test_var)))
    rmse_variance.append(np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    mae_variance.append(np.mean(abs(y_test - y_pred_test_var)))

Performing iteration : 0
Number of training data with variance: 7
Number of pooling data with variance: 63


Sample: 100%|██████████| 200/200 [02:58,  1.12it/s, step size=3.63e-03, acc. prob=0.114]


PCC_test variance 0.08758268225918316
R2_test variance -0.5672008323990254
RMSE 0.1545188335548614
MAE 0.10500195016676754
Performing iteration : 1
Number of training data with variance: 17
Number of pooling data with variance: 53


Sample: 100%|██████████| 200/200 [05:57,  1.79s/it, step size=3.73e-04, acc. prob=0.769]


PCC_test variance 0.8797113904821272
R2_test variance 0.7587203758547246
RMSE 0.0606288621980839
MAE 0.020404111764463242
Performing iteration : 2
Number of training data with variance: 27
Number of pooling data with variance: 43


...

Sample: 100%|██████████| 200/200 [04:55,  1.48s/it, step size=1.14e-04, acc. prob=0.539]


PCC_test variance 0.9769899822443827
R2_test variance 0.9432297902584679
RMSE 0.02940893812318504
MAE 0.007007963474344847
Performing iteration : 5
Number of training data with variance: 57
Number of pooling data with variance: 13


Sample: 100%|██████████| 200/200 [04:14,  1.27s/it, step size=2.30e-04, acc. prob=0.345]

PCC_test variance 0.976245361478677
R2_test variance 0.9529630217509167
RMSE 0.02676941011178182
MAE 0.0072710531602712325


'\n    # random sampling\n    if i != 0:\n        # select 10 random data points\n        q_points_ran = np.random.choice(np.arange(len(idx_pool_ran)), size=10)\n        # indices of those points in idx_pool\n        idx_pool_train_ran = idx_pool_ran[q_points_ran]\n\n        idx_train_ran = np.append(idx_train_ran, idx_pool_train_ran)\n        idx_pool_ran = np.delete(idx_pool_ran, q_points_ran)\n        X_train_ran = X[idx_train_ran]\n        y_train_ran = y[idx_train_ran]\n\n    print(f"Number of training data with random: {len(idx_train_ran)}")\n    print(f"Number of training data with random: {len(idx_pool_ran)}")\n\n\n    model = BNN(hid_dim=10, n_hid_layers=3, prior_scale=1)\n    nuts_kernel = NUTS(model, jit_compile=False)\n    mcmc = MCMC(nuts_kernel, num_samples=50)\n    mcmc.run(torch.Tensor(X_train_ran), torch.Tensor(y_train_ran))\n    predictive_ran = Predictive(model=model, posterior_samples=mcmc.get_samples())\n    preds_ran = predictive_ran(torch.Tensor(X_test))\n    # m

In [9]:
import pickle

with open('AL_BNN_MCMC_TI.pkl', 'wb') as f:

    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

pkl_file = open('AL_BNN_MCMC_TI.pkl', 'rb')  
test_ALGPR = pickle.load(pkl_file)